In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import time
import numpy as np
'''
import sys
sys.path.append('Classes')
from arpy import *
'''
import arpa
from tqdm import tqdm
import pickle
import torch

# Input - taken from set 1
### prefix count
### ngram count
### n-1gram count
### unigram count
### pre-pre fix

# Target - taken from set 2
### $P_{MLE}$


# check cuda


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(torch.cuda.get_device_name(0))
print(torch.cuda.is_available())

if device.type == 'cuda':
    #print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**2,1), 'MB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**2,1), 'MB')

GeForce GTX 1060 6GB
True
Memory Usage:
Allocated: 0.0 MB
Cached:    0.0 MB


# set 1 dictionary

In [3]:
file = '../../rsc/11_12_counts.txt'
first_read = open(file ,'r')

num_lines = sum(1 for line in open(file,'r'))

ngram_dict1 = {}
for x in tqdm(first_read, total=num_lines):
    line = x.split('\t')
    r = int(line[-1])
    ngram_dict1[line[0]] = r


100%|██████████| 32203379/32203379 [00:31<00:00, 1025549.36it/s]


# set 2 dictionary

In [4]:
file = '../../rsc/13_14_counts.txt'
first_read = open(file ,'r')

num_lines = sum(1 for line in open(file,'r'))

ngram_dict2 = {}
for x in tqdm(first_read, total=num_lines):
    line = x.split('\t')
    r = int(line[-1])
    ngram_dict2[line[0]] = r

100%|██████████| 28246110/28246110 [00:27<00:00, 1037756.01it/s]


# get tensor length

In [5]:
file = '../../rsc/11_12_counts.txt'
first_read = open(file ,'r')

num_lines = sum(1 for line in open(file,'r'))
num_trigrams = 0

for x in tqdm(first_read, total=num_lines, position = 0, leave = True):
    line = x.split('\t')
    r = int(line[-1])
    ngram = line[0].split(' ')
    tuple_size = len(ngram)

    if tuple_size == 3 and r < 8 and r!= 1:
        num_trigrams += 1
        
        try:
            #output - from set 1 = trigram count/ prefix
            temp = ngram_dict2[line[0]] / ngram_dict2[ngram[0] + ' ' + ngram[1]]
        except:
            num_trigrams -= 1
            
##########repeat for reverse set#########################
file = '../../rsc/13_14_counts.txt'
second_read = open(file ,'r')
num_lines = sum(1 for line in open(file,'r'))

for x in tqdm(second_read, total=num_lines, position = 0, leave = True):
    line = x.split('\t')
    r = int(line[-1])
    ngram = line[0].split(' ')
    tuple_size = len(ngram)

    if tuple_size == 3 and r < 8 and r!= 1:
        num_trigrams += 1
        
        try:
            #output - from set 1 = trigram count/ prefix
            temp = ngram_dict1[line[0]] / ngram_dict1[ngram[0] + ' ' + ngram[1]]
        except:
            num_trigrams -= 1


print(num_trigrams)

100%|██████████| 28246110/28246110 [00:29<00:00, 950612.45it/s]

4793918


# instantiate tensor

In [6]:
#inputs = data[0].to(device)

inputs = torch.zeros(5,num_trigrams, dtype=torch.float, device = device)
print(inputs)
outputs = torch.zeros(1,num_trigrams, dtype=torch.float, device = device)
print(outputs)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')


# fill tensors

In [7]:
file = '../../rsc/11_12_counts.txt'
first_read = open(file ,'r')

num_lines = sum(1 for line in open(file,'r'))
count = 0
err = 0

for x in tqdm(first_read, total=num_lines, position = 0, leave = True):
    line = x.split('\t')
    r = int(line[-1])
    ngram = line[0].split(' ')
    tuple_size = len(ngram)
    
    if tuple_size == 3 and r < 8 and r!= 1:
       
        try:
            #output - from set 2 = trigram count/ prefix
            outputs[0][count] = ngram_dict2[line[0]] / ngram_dict2[ngram[0] + ' ' + ngram[1]]
            
            #input - from set 1
            inputs[0][count] = ngram_dict1[ngram[0] + ' ' + ngram[1]]  #prefix count
            inputs[1][count] = ngram_dict1[line[0]]                    #trigram count
            inputs[2][count] = ngram_dict1[ngram[1] + ' ' + ngram[2]]  #backoff bigram count
            inputs[3][count] = ngram_dict1[ngram[2]]                   #unigram count
            inputs[4][count] = ngram_dict1[ngram[0]]                   #pre-pre fix

            count += 1

        except:
            err += 1
            
        

#######################now swap around sets and repeat#######################################
file = '../../rsc/13_14_counts.txt'
second_read = open(file ,'r')
num_lines = sum(1 for line in open(file,'r'))


for x in tqdm(second_read, total=num_lines, position = 0, leave = True):
    line = x.split('\t')
    r = int(line[-1])
    ngram = line[0].split(' ')
    tuple_size = len(ngram)
    
    if tuple_size == 3 and r < 8 and r!= 1:     
        
        try:
            #output - from set 1 = trigram count/ prefix
            outputs[0][count] = ngram_dict1[line[0]] / ngram_dict1[ngram[0] + ' ' + ngram[1]]
            
            #input - from set 2
            inputs[0][count] = ngram_dict2[ngram[0] + ' ' + ngram[1]]  #prefix count
            inputs[1][count] = ngram_dict2[line[0]]                    #trigram count
            inputs[2][count] = ngram_dict2[ngram[1] + ' ' + ngram[2]]  #backoff bigram count
            inputs[3][count] = ngram_dict2[ngram[2]]                   #unigram count
            inputs[4][count] = ngram_dict2[ngram[0]]                   #pre-pre fix

            count += 1

        except:
            err += 1
            
        
print(count)
print(err)

100%|██████████| 28246110/28246110 [07:51<00:00, 59943.94it/s] 

4793918
5331184


In [8]:
print(inputs)
print(outputs)

tensor([[1.6000e+01, 4.9000e+01, 4.9000e+01,  ..., 2.2000e+01, 4.2500e+02,
         4.2500e+02],
        [2.0000e+00, 4.0000e+00, 2.0000e+00,  ..., 3.0000e+00, 2.0000e+00,
         5.0000e+00],
        [3.8470e+03, 1.6500e+02, 1.3400e+02,  ..., 3.8400e+02, 3.0000e+00,
         4.8900e+02],
        [1.3060e+06, 2.0879e+04, 1.3656e+04,  ..., 8.9540e+03, 3.7470e+03,
         1.3817e+04],
        [8.0870e+03, 8.0870e+03, 8.0870e+03,  ..., 8.0870e+03, 1.3060e+06,
         1.3060e+06]], device='cuda:0')
tensor([[0.1538, 0.0667, 0.0667,  ..., 0.1429, 0.0118, 0.0118]],
       device='cuda:0')


In [8]:
print(inputs)
print(outputs)

tensor([[7.0000e+00, 6.0000e+00, 3.0000e+00,  ..., 4.0000e+00, 5.0000e+00,
         5.0000e+00],
        [2.0000e+00, 2.0000e+00, 3.0000e+00,  ..., 3.0000e+00, 3.0000e+00,
         2.0000e+00],
        [3.3570e+03, 5.4990e+03, 8.7100e+02,  ..., 1.5363e+04, 3.2000e+01,
         2.0000e+00],
        [1.2480e+05, 1.6983e+06, 4.6025e+04,  ..., 3.0418e+06, 2.2400e+04,
         1.4920e+03],
        [4.9700e+02, 4.9700e+02, 4.9700e+02,  ..., 1.0950e+03, 1.8000e+01,
         5.0000e+00]], device='cuda:0')
tensor([[1.0000, 0.1667, 1.0000,  ..., 0.6000, 1.0000, 0.1429]],
       device='cuda:0')


# Saving Tensors

In [9]:
torch.save(inputs, '../../pickles/50-50_5input_' + str(num_trigrams))
torch.save(outputs, '../../pickles/50-50_output_' + str(num_trigrams))